<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Evals that Work</h1>

Building great AI native products requires a rigorous evaluation process. While the idea of evaluation-driven development may seem novel to some, it really is the scientific method in disguise.  Just as scientists meticulously record experiments and take detailed notes to advance their understanding, AI systems require rigorous observation through tracing and annotations to reach their full potential. The goal of AI-native products is to build tools that empower humans, and it requires careful human judgment to align AI with human preferences and values.

<img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/scientific_method.png">

In [1]:
!pip install "arize-phoenix>=10.0.0" openai 'httpx<0.28' duckdb datasets pyarrow "pydantic>=2.0.0" nest_asyncio openinference-instrumentation-openai --quiet

This tutorial assumes you have a locally running Phoenix server. We can think of phoenix like a video recorder, observing every activity of your AI application.

```shell
phoenix serve
```

Let's also setup tracing for OpenAI as we will be using their API to perform the synthesis.

In [ ]:
from phoenix.otel import register

tracer_provider = register(
    project_name="basketball-app",
    auto_instrument=True, # Start recording traces via OpenAIInstrumentor
)

tracer = tracer_provider.get_tracer(__name__)

/Users/mikeldking/work/phoenix/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: basketball-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



Let's make sure we can run async code in the notebook.

In [3]:
import nest_asyncio

nest_asyncio.apply()

Lastly, let's make sure we have our openai API key set up.

In [4]:
import os
from getpass import getpass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

## Download Data

We are going to use the NBA dataset that information from 2014 - 2018. We will use DuckDB as our database.

In [5]:
import duckdb
from datasets import load_dataset

data = load_dataset("suzyanil/nba-data")["train"]

conn = duckdb.connect(database=":memory:", read_only=False)
conn.register("nba", data.to_pandas())

conn.query("SELECT * FROM nba LIMIT 5").to_df().to_dict(orient="records")[0]

{'Unnamed: 0': 1,
 'Team': 'ATL',
 'Game': 1,
 'Date': '10/29/14',
 'Home': 'Away',
 'Opponent': 'TOR',
 'WINorLOSS': 'L',
 'TeamPoints': 102,
 'OpponentPoints': 109,
 'FieldGoals': 40,
 'FieldGoalsAttempted': 80,
 'FieldGoals.': 0.5,
 'X3PointShots': 13,
 'X3PointShotsAttempted': 22,
 'X3PointShots.': 0.591,
 'FreeThrows': 9,
 'FreeThrowsAttempted': 17,
 'FreeThrows.': 0.529,
 'OffRebounds': 10,
 'TotalRebounds': 42,
 'Assists': 26,
 'Steals': 6,
 'Blocks': 8,
 'Turnovers': 17,
 'TotalFouls': 24,
 'Opp.FieldGoals': 37,
 'Opp.FieldGoalsAttempted': 90,
 'Opp.FieldGoals.': 0.411,
 'Opp.3PointShots': 8,
 'Opp.3PointShotsAttempted': 26,
 'Opp.3PointShots.': 0.308,
 'Opp.FreeThrows': 27,
 'Opp.FreeThrowsAttempted': 33,
 'Opp.FreeThrows.': 0.818,
 'Opp.OffRebounds': 16,
 'Opp.TotalRebounds': 48,
 'Opp.Assists': 26,
 'Opp.Steals': 13,
 'Opp.Blocks': 9,
 'Opp.Turnovers': 9,
 'Opp.TotalFouls': 22}

## Implement Text2SQL

Let's start by implementing a simple text2sql logic.

In [ ]:
import os

import openai

from phoenix.client import Client
from phoenix.client.types import PromptVersion

phoenix_client = Client()
client = openai.AsyncClient()

columns = conn.query("DESCRIBE nba").to_df().to_dict(orient="records")

# We will use GPT4o to start
TASK_MODEL = "gpt-4o"
CONFIG = {"model": TASK_MODEL}

system_prompt = (
    "You are a SQL expert, and you are given a single table named nba with the following columns:\n"
    f'{",".join(column["column_name"] + ": " + column["column_type"] for column in columns)}\n'
    "Write a SQL query corresponding to the user's request. Return just the query text, "
    "with no formatting (backticks, markdown, etc.)."
)

prompt_template = phoenix_client.prompts.create(
    name="text2sql",
    version=PromptVersion(
        [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": "{{input}}",
            },
        ],
        model_name=TASK_MODEL,
    ),
)



@tracer.chain
async def generate_query(input):
    prompt = prompt_template.format(variables={"input": input}, sdk="openai")
    response = await client.chat.completions.create(
        **prompt,
        temperature=0,
    )
    return response.choices[0].message.content

In [8]:
query = await generate_query("Who won the most games?")
print(query)

SELECT Team, COUNT(*) AS Wins
FROM nba
WHERE WINorLOSS = 'W'
GROUP BY Team
ORDER BY Wins DESC
LIMIT 1;


Awesome, looks like the LLM is producing SQL! let's try running the query and see if we get the expected results.

In [10]:
@tracer.chain
def execute_query(query):
    return conn.query(query).fetchdf().to_dict(orient="records")


execute_query(query)

[{'Team': 'GSW', 'Wins': 265}]

Let's put the pieces together and see if we can create a basketball agent

In [25]:
@tracer.chain
async def text2sql(question):
    query = await generate_query(question)
    results = None
    error = None
    try:
        results = execute_query(query)
    except duckdb.Error as e:
        error = str(e)

    return {
        "query": query,
        "results": results,
        "error": error,
    }

@tracer.agent
async def basketball_agent(question):
    sql_response = await text2sql(question)
    if sql_response["error"]:
        raise Exception(sql_response["error"])
    results = sql_response["results"]
    answer = await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": f"The results of the query are: {results}. Answer the following question: {question}"},
        ],
    )
    return answer.choices[0].message.content

await basketball_agent("Who won the most games?")


Transient error StatusCode.UNAVAILABLE encountered while exporting traces to localhost:4317, retrying in 1s.


'The team that won the most games is GSW with 265 wins.'

Let's run the agent over some examples

In [27]:
from phoenix.trace import using_project

questions = [
    "Which team won the most games?",
    "Which team won the most games in 2015?",
    "Who led the league in 3 point shots?",
    "Which team had the biggest difference in records across two consecutive years?",
    "What is the average number of free throws per year?",
]

with using_project(project_name="basketball-agent-test"):
    for question in questions:
        try:
            answer = await basketball_agent(question)
            print(answer)
        except Exception as e:
            print(e)



The team that won the most games is the Golden State Warriors (GSW) with 265 wins.
It seems there are no results from the query you provided. However, if you're asking about which Major League Baseball team won the most games in the 2015 season, the answer is the St. Louis Cardinals. They finished the regular season with a total of 100 wins, the most in MLB that year.
The team that led the league in 3-point shots, based on the given query result, is the Houston Rockets (HOU), with a total of 4,248 three-point shots.
Binder Error: column "Team" must appear in the GROUP BY clause or must be part of an aggregate function.
Either add it to the GROUP BY list, or use "ANY_VALUE(Team)" if the exact value of "Team" is not important.
To find the average number of free throws per year, we need to calculate the mean of all the provided `AverageFreeThrows` values from the list. Here's how you can approach it:

1. Sum the `AverageFreeThrows` values.
2. Divide the total by the number of entries to d

Let's look at the data and annotate some of the data to see what the issues might be. Once we've annotated some data we can bring it back into the notebook to analyze it.

In [33]:
from phoenix.client import Client
from phoenix.trace.dsl import SpanQuery

phoenix_client = Client()
query = SpanQuery().where("name == 'basketball_agent'")

spans_df = phoenix_client.spans.get_spans_dataframe(project_identifier="basketball-agent-test", query=query)
annotations_df = phoenix_client.spans.get_span_annotations_dataframe(
    spans_dataframe=spans_df, project_identifier="basketball-agent-test"
)

annotations_df.join(spans_df, how="inner")



,annotation_name,annotator_kind,metadata,identifier,id,created_at,updated_at,source,user_id,result.label,...,status_code,status_message,events,context.span_id,context.trace_id,attributes.output.value,attributes.input.value,attributes.input.mime_type,attributes.openinference.span.kind,attributes.output.mime_type
span_id,,,,,,,,,,,,,,,,,,,,,
d84ac33961fca54d,correctness,HUMAN,{},,U3BhbkFubm90YXRpb246MTQ=,2025-05-24T00:56:13+00:00,2025-05-24T00:56:13+00:00,APP,None,correct,...,OK,,[],d84ac33961fca54d,7b1908e8a34ad4ad872fd87fb0ff51a2,The team that won the most games is the Golden...,Which team won the most games?,text/plain,AGENT,text/plain
a2189438310eefe3,correctness,HUMAN,{},,U3BhbkFubm90YXRpb246MTM=,2025-05-24T00:56:07+00:00,2025-05-24T00:56:07+00:00,APP,None,incorrect,...,OK,,[],a2189438310eefe3,614e3f82e07552fe43345ac2a1cd7658,The information provided does not include any ...,Which team won the most games in 2015?,text/plain,AGENT,text/plain
17ab342444da91d1,correctness,HUMAN,{},,U3BhbkFubm90YXRpb246MTI=,2025-05-24T00:56:03+00:00,2025-05-24T00:56:03+00:00,APP,None,correct,...,OK,,[],17ab342444da91d1,6c99d120d450daa11cbefb6048889faf,"Based on the query results you provided, the t...",Who led the league in 3 point shots?,text/plain,AGENT,text/plain
9b6973ac759dddca,correctness,HUMAN,{},,U3BhbkFubm90YXRpb246MTE=,2025-05-24T00:55:57+00:00,2025-05-24T00:55:57+00:00,APP,None,incorrect,...,ERROR,"Exception: Binder Error: column ""Team"" must ap...","[{'name': 'exception', 'timestamp': '2025-05-2...",9b6973ac759dddca,eec9a0612a4ae9fb60f2ddab67005697,None,Which team had the biggest difference in recor...,text/plain,AGENT,None
e545c971b4bddb0c,correctness,HUMAN,{},,U3BhbkFubm90YXRpb246MTA=,2025-05-24T00:55:47+00:00,2025-05-24T00:55:47+00:00,APP,None,correct,...,OK,,[],e545c971b4bddb0c,7c6e7b7e87bb93b00c552ee948d00353,The team that won the most games is the Golden...,Which team won the most games?,text/plain,AGENT,text/plain
b2ebaff0f5058e3d,note,HUMAN,{},px-span-note:2025-05-23T18:54:40.416838,U3BhbkFubm90YXRpb246Nw==,2025-05-24T00:54:40+00:00,2025-05-24T00:54:40+00:00,APP,None,None,...,OK,,[],b2ebaff0f5058e3d,61b175558ed8db089985dee069080fbe,It seems there are no results from the query y...,Which team won the most games in 2015?,text/plain,AGENT,text/plain
b2ebaff0f5058e3d,correctness,HUMAN,{},,U3BhbkFubm90YXRpb246Ng==,2025-05-24T00:54:32+00:00,2025-05-24T00:54:32+00:00,APP,None,incorrect,...,OK,,[],b2ebaff0f5058e3d,61b175558ed8db089985dee069080fbe,It seems there are no results from the query y...,Which team won the most games in 2015?,text/plain,AGENT,text/plain
d8d9aa8a9a4b79ea,correctness,HUMAN,{},,U3BhbkFubm90YXRpb246NQ==,2025-05-24T00:53:45+00:00,2025-05-24T00:58:35+00:00,APP,None,correct,...,OK,,[],d8d9aa8a9a4b79ea,47c83b3f507f02403cd13063d347c858,"The team that led the league in 3-point shots,...",Who led the league in 3 point shots?,text/plain,AGENT,text/plain
b54c0476116fadf5,correctness,HUMAN,{},,U3BhbkFubm90YXRpb246NA==,2025-05-24T00:53:36+00:00,2025-05-24T00:53:36+00:00,APP,None,incorrect,...,ERROR,"Exception: Binder Error: column ""Team"" must ap...","[{'name': 'exception', 'timestamp': '2025-05-2...",b54c0476116fadf5,69aed5b204f7597c2bc4ed1cebabe5b0,None,Which team had the biggest difference in recor...,text/plain,AGENT,None


## Evaluation

<img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/evaluator.png">
Evaluation consists of three parts — data, task, and scores. We'll start with data.

In [11]:
questions = [
    "Which team won the most games?",
    "Which team won the most games in 2015?",
    "Who led the league in 3 point shots?",
    "Which team had the biggest difference in records across two consecutive years?",
    "What is the average number of free throws per year?",
]

Let's store the data above as a versioned dataset in phoenix.

In [34]:
import phoenix as px
import pandas as pd

ds = px.Client().upload_dataset(
    dataset_name="nba-questions-v2",
    dataframe=pd.DataFrame([{"question": question} for question in questions]),
    input_keys=["question"],
    output_keys=[],
)

# If you have already uploaded the dataset, you can fetch it using the following line
# ds = px.Client().get_dataset(name="nba-questions")

📤 Uploading dataset...
💾 Examples uploaded: http://127.0.0.1:6006/datasets/RGF0YXNldDoyOA==/examples
🗄️ Dataset version ID: RGF0YXNldFZlcnNpb246Mjg=


Next, we'll define the task. The task is to generate SQL queries from natural language questions.

In [35]:
@tracer.chain
async def text2sql(question):
    query = await generate_query(question)
    results = None
    error = None
    try:
        results = execute_query(query)
    except duckdb.Error as e:
        error = str(e)

    return {
        "query": query,
        "results": results,
        "error": error,
    }

Finally, we'll define the scores. We'll use the following simple scoring functions to see if the generated SQL queries are correct.

In [36]:
# Test if there are no sql execution errors

def no_error(output):
    return 1.0 if output.get("error") is None else 0.0


# Test if the query has results
def has_results(output):
    results = output.get("results")
    has_results = results is not None and len(results) > 0
    return 1.0 if has_results else 0.0

Now let's run the evaluation experiment.

In [ ]:
import phoenix as px
from phoenix.experiments import run_experiment


# Define the task to run text2sql on the input question
def task(input):
    return text2sql(input["question"])


experiment = run_experiment(
    ds, task=task, evaluators=[no_error, has_results], experiment_metadata=CONFIG
)

🧪 Experiment started.
📺 View dataset experiments: http://127.0.0.1:6006/datasets/RGF0YXNldDoyOA==/experiments
🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDoyOA==/compare?experimentId=RXhwZXJpbWVudDozNg==


running tasks |          | 0/5 (0.0%) | ⏳ 00:00<? | ?it/sTransient error StatusCode.UNAVAILABLE encountered while exporting traces to localhost:4317, retrying in 1s.
running tasks |██████████| 5/5 (100.0%) | ⏳ 00:03<00:00 |  1.52it/s

✅ Task runs completed.
🧠 Evaluation started.


running tasks |██████████| 5/5 (100.0%) | ⏳ 00:04<00:00 |  1.04it/s



🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDoyOA==/compare?experimentId=RXhwZXJpbWVudDozNg==

Experiment Summary (05/23/25 07:15 PM -0600)
--------------------------------------------
     evaluator  n  n_scores  avg_score
0  has_results  5         5        0.6
1     no_error  5         5        0.8

Tasks Summary (05/23/25 07:15 PM -0600)
---------------------------------------
   n_examples  n_runs  n_errors
0           5       5         0


Ok! It looks like 3/5 of our queries are valid.


## Interpreting the results

Now that we ran the initial evaluation, it looks like two of the results are valid, two produce SQL errors, and one is incorrect.

- The incorrect query didn't seem to get the date format correct. That would probably be improved by showing a sample of the data to the model (e.g. few shot example).

- There are is a binder error, which may also have to do with not understanding the data format.

Let's try to improve the prompt with few-shot examples and see if we can get better results.

In [ ]:
samples = conn.query("SELECT * FROM nba LIMIT 1").to_df().to_dict(orient="records")[0]
sample_rows = "\n".join(
    f"{column['column_name']} | {column['column_type']} | {samples[column['column_name']]}"
    for column in columns
)
system_prompt = (
    "You are a SQL expert, and you are given a single table named nba with the following columns:\n\n"
    "Column | Type | Example\n"
    "-------|------|--------\n"
    f"{sample_rows}\n"
    "\n"
    "Write a DuckDB SQL query corresponding to the user's request. "
    "Return just the query text, with no formatting (backticks, markdown, etc.)."
)


async def generate_query(input):
    response = await client.chat.completions.create(
        model=TASK_MODEL,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )
    return response.choices[0].message.content


print(await generate_query("Which team won the most games in 2015?"))

Looking much better! Finally, let's add a scoring function that compares the results, if they exist, with the expected results.




In [ ]:
experiment = run_experiment(
    ds, task=task, evaluators=[has_results, no_error], experiment_metadata=CONFIG
)

Amazing. It looks like we removed one of the errors, and got a result for the incorrect query. Let's try out using LLM as a judge to see how well it can assess the results.


In [ ]:
from phoenix.evals.models import OpenAIModel
from phoenix.experiments import evaluate_experiment
from phoenix.experiments.evaluators.llm_evaluators import LLMCriteriaEvaluator

llm_evaluator = LLMCriteriaEvaluator(
    name="is_sql",
    criteria="is_sql",
    description="the output is a valid SQL query and that it executes without errors",
    model=OpenAIModel(),
)

evaluate_experiment(experiment, evaluators=[llm_evaluator])

Sure enough the LLM agrees with our scoring. Pretty neat trick! This can come in useful when it's difficult to define a scoring function.


We now have a simple text2sql pipeline that can be used to generate SQL queries from natural language questions. Since Phoenix has been tracing the entire pipeline, we can now use the Phoenix UI to convert the spans that generated successful queries into examples to use in **Golden Dataset** for regression testing!

<img src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/golden_dataset.png">

## Generating more data
Now that we have a basic flow in place, let's generate some data. We're going to use the dataset itself to generate expected queries, and have a model describe the queries. This is a slightly more robust method than having it generate queries, because we'd expect a model to describe a query more accurately than generate one from scratch.




In [ ]:
import json
from typing import List

from pydantic import BaseModel


class Question(BaseModel):
    sql: str
    question: str


class Questions(BaseModel):
    questions: List[Question]


sample_rows = "\n".join(
    f"{column['column_name']} | {column['column_type']} | {samples[column['column_name']]}"
    for column in columns
)
synthetic_data_prompt = f"""You are a SQL expert, and you are given a single table named nba with the following columns:

Column | Type | Example
-------|------|--------
{sample_rows}

Generate SQL queries that would be interesting to ask about this table. Return the SQL query as a string, as well as the
question that the query answers."""

response = await client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": synthetic_data_prompt,
        }
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "generate_questions",
                "description": "Generate SQL queries that would be interesting to ask about this table.",
                "parameters": Questions.model_json_schema(),
            },
        }
    ],
    tool_choice={"type": "function", "function": {"name": "generate_questions"}},
)

generated_questions = json.loads(response.choices[0].message.tool_calls[0].function.arguments)[
    "questions"
]
generated_questions[0]

In [ ]:
generated_dataset = []
for q in generated_questions:
    try:
        result = execute_query(q["sql"])
        generated_dataset.append(
            {
                "input": q["question"],
                "expected": {
                    "results": result,
                    "error": None,
                    "query": q["sql"],
                },
                "metadata": {
                    "category": "Generated",
                },
            }
        )
    except duckdb.Error as e:
        print(f"Query failed: {q['sql']}", e)
        print("Skipping...")

generated_dataset[0]

Awesome, let's crate a dataset with the new synthetic data.




In [ ]:
synthetic_dataset = px.Client().upload_dataset(
    dataset_name="nba-golden-synthetic",
    inputs=[{"question": example["input"]} for example in generated_dataset],
    outputs=[example["expected"] for example in generated_dataset],
);

In [ ]:
run_experiment(
    synthetic_dataset, task=task, evaluators=[no_error, has_results], experiment_metadata=CONFIG
)

Amazing! Now we have a rich dataset to work with and some failures to debug. From here, you could try to investigate whether some of the generated data needs improvement, or try tweaking the prompt to improve accuracy, or maybe even something more adventurous, like feed the errors back to the model and have it iterate on a better query. Most importantly, we have a good workflow in place to iterate on both the application and dataset.



# Trying a smaller model
Just for fun, let's wrap things up by trying out GPT-3.5-turbo. All we need to do is switch the model name, and run our Eval() function again.




In [ ]:
TASK_MODEL = "gpt-3.5-turbo"

experiment = run_experiment(
    synthetic_dataset,
    task=task,
    evaluators=[no_error, has_results],
    experiment_metadata={"model": TASK_MODEL},
)

Interesting! It looks like the smaller model is able to do decently well but we might want to ensure it follows instructions as well as a larger model. We can actually grab all the LLM spans from our previous GPT40 runs and use them to generate a OpenAI fine-tuning JSONL file!

<img src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/fine_tining_nba.png">
<img src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/openai_ft.png">

## Conclusion

In this example, we walked through the process of building a dataset for a text2sql application. We started with a few handwritten examples, and iterated on the dataset by using an LLM to generate more examples. We used the eval framework to track our progress, and iterated on the model and dataset to improve the results. Finally, we tried out a less powerful model to see if we could save cost or improve latency.

Happy evaluations!